In [1]:
import torch
from tqdm.notebook import tqdm


from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification


In [2]:
import pandas as pd
df = pd.read_csv('processed_lyrics.csv')


In [3]:
df

,ALink,SName,SLink,Lyric,Idiom,Genre,Artist
0,/12-stones/,World So Cold,/12-stones/world-so-cold.html,"It starts with pain, followed by hate. Fueled ...",ENGLISH,Rock,12 Stones
1,/12-stones/,Broken,/12-stones/broken.html,Freedom!. Alone again again alone. Patiently w...,ENGLISH,Rock,12 Stones
2,/12-stones/,3 Leaf Loser,/12-stones/3-leaf-loser.html,"Biting the hand that feeds you, lying to the v...",ENGLISH,Rock,12 Stones
3,/12-stones/,Anthem For The Underdog,/12-stones/anthem-for-the-underdog.html,You say you know just who I am. But you can't ...,ENGLISH,Rock,12 Stones
4,/12-stones/,Adrenaline,/12-stones/adrenaline.html,My heart is beating faster can't control these...,ENGLISH,Rock,12 Stones
...,...,...,...,...,...,...,...
94809,/zendaya/,Too Much,/zendaya/too-much.html,"Baby we're too much, much to handle. Understan...",ENGLISH,Pop,Zendaya
94810,/zendaya/,Watch Me (feat. Bella Thorne),/zendaya/watch-me-feat-bella-thorne.html,I don't need no one to tell me how to feel. Th...,ENGLISH,Pop,Zendaya
94811,/zendaya/,White Christmas,/zendaya/white-christmas.html,"I'm, dreaming of a white, Christmas. Just like...",ENGLISH,Pop,Zendaya
94812,/zendaya/,Whodunit,/zendaya/whodunit.html,"Ha, a-plus on the track. Ha, ha, ha, uh, yeah....",ENGLISH,Pop,Zendaya


In [4]:
subset = df.sample(n=1000)

In [5]:
subset


,ALink,SName,SLink,Lyric,Idiom,Genre,Artist
75663,/david-archuleta/,Shine a Light,/david-archuleta/shine-a-light.html,There is a beacon. You might not see the light...,ENGLISH,Pop,David Archuleta
45819,/the-go-team/,Doing It Right,/the-go-team/doing-it-right.html,Do it! Do it! All right!. Do it! Do it! All ri...,ENGLISH,Rock,The Go! Team
21810,/killing-joke/,Absent Friends,/killing-joke/absent-friends.html,I remember the sun beating down. Across my bac...,ENGLISH,Rock,Killing Joke
70872,/bee-gees/,Woman In Love,/bee-gees/woman-in-love.html,Life is a moment in space. When the dream is g...,ENGLISH,Pop,Bee Gees
84756,/madonna/,Live To Tell,/madonna/live-to-tell.html,I have a tale to tell.. Sometimes it gets so h...,ENGLISH,Pop,Madonna
...,...,...,...,...,...,...,...
64416,/snoop-dogg/,Speaker,/snoop-dogg/speaker.html,"[Intro]. Snoop Dogg. Konvict Music, Akon and D...",ENGLISH,Hip Hop,Snoop Dogg
85770,/michael-jackson/,Seeing Voices,/michael-jackson/seeing-voices.html,I sing of what you see and might think about. ...,ENGLISH,Pop,Michael Jackson
71975,/bread/,Diary,/bread/diary.html,I found her diary underneath a tree. and start...,ENGLISH,Pop,Bread
75732,/david-guetta/,Bang My Head (feat. Sia & Fetty Wap) (Club Kil...,/david-guetta/bang-my-head-feat-sia-fetty-wap-...,"I was bound, was tired. Hadn't seen a light so...",ENGLISH,Pop,David Guetta


In [8]:
subset['Genre'].value_counts()

Rock       563
Pop        286
Hip Hop    151
Name: Genre, dtype: int64

Since the Genres are imbalanced we use this as 

In [9]:

possible_labels = df.Genre.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Rock': 0, 'Pop': 1, 'Hip Hop': 2}

In [10]:
subset['label'] = subset.Genre.replace(label_dict)

In [11]:
from sklearn.model_selection import train_test_split
 
X_train, X_val, y_train, y_val = train_test_split(
    subset.index.values,
    subset.label.values,
    test_size = 0.2,
    random_state=42,
    stratify=subset.label.values
)

In [12]:
X_train

array([90614, 66945, 11708, 56317, 78223, 83365, 35499, 28693, 42136,
       77595, 82675, 85607, 42509, 11460, 38540, 68823, 43948, 34780,
       70202, 93038, 81668, 92592,  5803, 26093, 38340, 13319,  3123,
       29371, 71975, 58258, 74168, 11603, 14949, 57364,  1029,  6540,
       75878,  8242, 32246,  5933, 24322, 17676, 58433, 46648, 87754,
       93211, 11051,    57,  6401,  9495, 76535, 57077, 81699, 56343,
       94031, 26385, 20513, 53614, 71550, 52168, 75855, 13094, 32235,
       23422, 78795,  9325, 52996, 81989, 42398, 22779,  9785,  9257,
       78340, 39703, 71360, 21262, 42746, 72696, 89887, 46491, 70973,
       32254, 14785, 38463, 66455, 60751, 68554, 28422,  3436, 28777,
       64982, 18829, 89829, 51070, 25790, 50182, 26395, 87178, 38301,
       14446, 63232, 53849, 25458, 90721, 11448,  5661, 62158,  3003,
       42821, 46794, 27810, 64416, 42332, 91261, 49698, 83676, 13608,
       63962, 87522, 70902, 79612, 94246, 84326, 45819, 57618, 85770,
       39130, 84064,

In [13]:
['not_set']*subset.shape[0]

['not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'not_set',
 'no

In [20]:
subset=subset.drop(['ALink', 'SName', 'SLink', 'Idiom', 'Artist'], axis=1)

In [21]:
subset

,Lyric,Genre,label
75663,There is a beacon. You might not see the light...,Pop,1
45819,Do it! Do it! All right!. Do it! Do it! All ri...,Rock,0
21810,I remember the sun beating down. Across my bac...,Rock,0
70872,Life is a moment in space. When the dream is g...,Pop,1
84756,I have a tale to tell.. Sometimes it gets so h...,Pop,1
...,...,...,...
64416,"[Intro]. Snoop Dogg. Konvict Music, Akon and D...",Hip Hop,2
85770,I sing of what you see and might think about. ...,Pop,1
71975,I found her diary underneath a tree. and start...,Pop,1
75732,"I was bound, was tired. Hadn't seen a light so...",Pop,1


In [22]:
#Adding a column with 'not_set'
subset['data_type'] = ['not_set']*subset.shape[0]

#Setting the train/validation values
subset.loc[X_train, 'data_type'] = 'train'
subset.loc[X_val, 'data_type'] = 'val'

subset.groupby(['Genre', 'label', 'data_type']).count()

Lyric
Genre   label data_type       
Hip Hop 2     train        121
              val           30
Pop     1     train        229
              val           57
Rock    0     train        450
              val          113

In [12]:

## TOKENIZE THE DATA
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    subset[subset.data_type=='train'].Lyric.values, 
    add_special_tokens=True,  # sequences will be encoded with the special tokens relative to their model.
    return_attention_mask=True, # batching sequences together. will return the attention mask according to the specific tokenizer defined by the max_length attribute.
    pad_to_max_length=True, # pad all the titles to certain maximum length
    max_length=256, 
    return_tensors='pt' # return PyTorch
)

encoded_data_val = tokenizer.batch_encode_plus(
    subset[subset.data_type=='val'].Lyric.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/ubuntu/miniconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2221: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [13]:
# split the data into input_ids, attention_masks and labels.
# after we get encoded data set, we can create training data and validation data.
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(subset[subset.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(subset[subset.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

### BERT Pre-trained Model
We are treating each Genre as its unique sequence, so one sequence will be classified to one of the three labels

In [14]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", # smaller pre-trained model.
                                                      num_labels=len(label_dict), # number of output labels.
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### Data Loaders 
DataLoader combines a dataset and a sampler and provides an iterable over the given dataset
RandomSampler is used for training and SequentialSampler for validation.
Batch_size is set after the available memory. 
#### Start with batch_size=3? Increase when using cloud computing

In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

# Optimizer & Scheduler
- To construct an optimizer, we have to give it an iterable containing the parameters to optimize. Then, we can specify optimizer-specific options such as the learning rate, epsilon, etc.
- Try with 5 epochs and iterate
- Create a schedule with a learning rate that decreases linearly from the initial learning rate set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial learning rate set in the optimizer.

In [16]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Performance Metrics
We will use f1 score and accuracy per class as performance metrics.


In [17]:

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [18]:
torch.cuda.is_available()

True

# Training Loop

In [21]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# In case you are running on gpu, you also need to set the model: model.to(device) - Is this set in the next step? 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/home/ubuntu/NLP{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/267 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.0026302170937127444
Validation loss: 0.0009898714279867153
F1 Score (Weighted): 1.0


Epoch 2:   0%|          | 0/267 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.0012140879252740646
Validation loss: 0.0005694024131021727
F1 Score (Weighted): 1.0


...


Epoch 3
Training loss: 0.0008274260876754348
Validation loss: 0.00043676700312713745
F1 Score (Weighted): 1.0


Epoch 4:   0%|          | 0/267 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.0007003494651730029
Validation loss: 0.00040270269365369607
F1 Score (Weighted): 1.0


Epoch 5:   0%|          | 0/267 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.0006763311243600348
Validation loss: 0.00040270269365369607
F1 Score (Weighted): 1.0


# Loading and Evaluating the Model

In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('/home/ubuntu/NLP5.model', map_location=torch.device('cuda')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: Rock
Accuracy: 105/105

Class: Pop
Accuracy: 61/61

Class: Hip Hop
Accuracy: 34/34

